# Named Entity Recognition without labelled data: A weak supervision approach

In this walkthrough, we will develop a neural NER model without access to labelled data. Let's look at a particular example of text (from Reuters):


In [2]:
import sys
sys.path.insert(0, '..')

In [5]:
import re
news_text  = """
ATLANTA  (Reuters) - Retailer Best Buy Co, seeking new ways to appeal to cost-conscious shoppers, said on Tuesday it is selling refurbished 
 versions of Apple Inc's iPhone 3G at its stores that are priced about $50 less than new iPhones. 
 The electronics chain said the used iPhones, which were returned within 30 days of purchase, are priced at $149 for the model with 8 gigabytes of storage, 
 while the 16-gigabyte version is $249. A two-year service contract with AT&T Inc is required. New iPhone 3Gs currently sell for $199 and $299 at 
 Best Buy Mobile stores. "This is focusing on customers' needs, trying to provide as wide a range of products and networks for our consumers," said 
 Scott Moore, vice president of marketing for Best Buy Mobile. Buyers of first-generation iPhones can also upgrade to the faster refurbished 3G models at 
 Best Buy, he said. Moore said AT&T, the exclusive wireless provider for the iPhone, offers refurbished iPhones online. The sale of used iPhones comes as 
 Best Buy, the top consumer electronics chain, seeks ways to fend off increased competition from discounters such as Wal-Mart Stores Inc, which began 
 selling the popular phone late last month. Wal-Mart sells a new 8-gigabyte iPhone 3G for $197 and $297 for the 16-gigabyte model. The iPhone is also 
 sold at Apple stores and AT&T stores. Moore said Best Buy's move was not in response to other retailers' actions. (Reporting by  Karen Jacobs ; Editing 
 by  Andre Grenon )"""

news_text = re.sub('\\s+', ' ', news_text)

Let's view what the standard Spacy model produces (note it takes a few seconds to reload the vocabulary):

In [38]:
import spacy, skweak

# We load the spacy model
nlp = spacy.load("en")
doc = nlp(news_text)

# Visualising the entities
skweak.utils.display_entities(doc)

The medium-size model works better, but still contains quite a few errors and omissions:

In [6]:

# We load the spacy model (takes a few seconds)
nlp = spacy.load("en_core_web_md")
doc = nlp(news_text)

# Visualising the entities
skweak.utils.display_entities(doc)

Ideally, one would wish to train a better named entity recognition model, which is better tailored to the specific needs and linguistic patterns found in those articles. However, although we have large amounts of raw text data, we often do not have text data labelled with named entities for our domain. We therefore worked on an alternative approach based on __weak supervision__, combining several (noisy) supervision sources instead of a single "gold standard". 

Indeed, we do have access to several possible supervision sources, such as alternative NER models trained on other corpora, large lists of entity (companies, person names, geographical locations), shallow linguistic patterns, and document-level constraints. 

The key idea behind the proposed approach is thus to (1) use these supervision sources to automatically annotate news corpora, (2) estimate a label model (more precisely an HMM model) that unifies all these sources into a single one, and (3) learn a new NER model based on these unified labels. <br>

__Outline of this notebook__: I describe below the various annotation schemes that I developed.  I then explain how these various sources can be merged into a single source using the `skweak` framework. Finally, I detail the architecture behind the NER model.

## __Step 1:__ Annotations

### 1) Annotators from other Spacy models

A first source of automatic annotation comes from NER models trained on multiple, distinct corpora. I went through [available NE-labelled corpora](https://github.com/juand-r/entity-recognition-datasets) to search for datasets that could be used to train alternative models. I then trained Spacy models for all of them, and then conducted some experiments to assess their performance. At the end of the process, I ended up with four models:
- The standard Spacy model for English (`en_core_web_md`), trained on Ontonotes v5
- A model trained on [ConLL 2003](https://www.clips.uantwerpen.be/conll2003/ner/)
- A model trained on the [Broad Twitter Corpus](https://github.com/GateNLP/broad_twitter_corpus)
- A model trained on a corpus of [SEC filings](https://www.aclweb.org/anthology/U15-1010/).
    
Note there are differences between the entity labels of these models: while Ontonotes contains no less than [18 classes](https://spacy.io/api/annotation#named-entities), the other corpora only contain `PER(SON)`, `ORG`, `LOC` and `MISC`. Furthermore, the labels also do not match each other perfectly: while Ontonotes distinguishes between geopolitical locations (`GPE`) and "natural" locations (such as continents, seas etc., labelled as `LOC`), the three other models regroup all geographical entities as `LOC`. 

We can apply annotations from a Spacy model using the `ModelAnnotator` class

In [5]:
annotator = skweak.base.ModelAnnotator("conll2003", "./data/conll2003/")

doc = annotator(doc)
skweak.utils.display_entities(doc, "conll2003")

As we can see, the results are not perfect on this model either, but the errors are distinct from the ones made by the Ontonotes model. 

The annotations are written in the `user_data` dictionary of the Spacy document:

In [6]:
doc.user_data["spans"]["conll2003"]

{(3, 4): 'ORG',
 (7, 10): 'ORG',
 (31, 33): 'ORG',
 (34, 35): 'ORG',
 (49, 50): 'ORG',
 (57, 58): 'ORG',
 (97, 99): 'ORG',
 (102, 104): 'ORG',
 (114, 117): 'ORG',
 (145, 147): 'PER',
 (153, 156): 'ORG',
 (162, 163): 'ORG',
 (174, 176): 'ORG',
 (180, 181): 'PER',
 (182, 183): 'PER',
 (190, 191): 'ORG',
 (194, 195): 'ORG',
 (201, 202): 'ORG',
 (204, 206): 'ORG',
 (224, 225): 'ORG',
 (226, 229): 'ORG',
 (240, 241): 'LOC',
 (242, 243): 'LOC',
 (247, 248): 'ORG',
 (262, 263): 'ORG',
 (267, 268): 'ORG',
 (273, 274): 'PER',
 (275, 277): 'ORG',
 (292, 294): 'PER',
 (297, 299): 'PER'}

Each `ModelAnnotator` adds two annotation sources: one that is directly based on the Spacy Model (here `conll2003`), and one that also includes the corrections specified in the method `_correct_entities` (in `spacy_wrapper.py`) that we implemented earlier this year.  The corrected version are indicated with a `+c` suffix.

Here are the results from the three other models:

In [7]:
core_web_annotator = skweak.base.ModelAnnotator("core_web_md", "en_core_web_md")

doc = core_web_annotator(doc)
skweak.utils.display_entities(doc, "core_web_md")


__Note__: When annotating large collections of news documents, the method `annotator.pipe(news_docs)` is much more efficient than calling `annotate(...)` every single time, as it batches the documents on which to run the NER model.

### 2) Annotators from gazetteers

Another useful source of annotation comes from large lists of entities such as persons, places and organisations. The gazetteers are using a _trie_ to efficiently search for occurrences in the text. Each gazetteer creates two annotation sources: one that is case-sensitive (`_cased` suffix) and one case-insentitive (`_uncased` suffix).


#### 2.1) Wikipedia
The database from Wikipedia is extracted from the [NECKar](https://event.ifi.uni-heidelberg.de/?page_id=532) dataset.  The postprocessing (which, among others, filters out entities that are also relatively common English words) is implemented in `compile_wikidata`. In addition, I also extracted from Wikidata a list of commercial products and added them to the gazetteer. 

In [7]:
import domains.conll2003_ner

tries = skweak.gazetteers.extract_json_data(domains.conll2003_ner.WIKIDATA)
annotator = skweak.gazetteers.GazetteerAnnotators("wiki", tries)

annotator(doc)
skweak.utils.display_entities(doc, "wiki_*")

Extracting data from ./data/wikidata.json
Populating trie for class PERSON (number: 2626732)
Populating trie for class LOC (number: 47129)
Populating trie for class GPE (number: 602953)
Populating trie for class ORG (number: 295766)
Populating trie for class PRODUCT (number: 12457)


Again, the annotation model does make some errors: `Moore` is thought to be a [geopolitical entity](https://en.wikipedia.org/wiki/Moore) instead of a person. Note that `AT&T` has two alternative labels: `ORG` or `GPE` (see [AT&T station](https://en.wikipedia.org/wiki/AT%26T_(SEPTA_station))).  

In addition to the full wiki data, I also added a specific gazetteer that only employs wikidata objects containing a text description:

In [8]:

tries = skweak.gazetteers.extract_json_data(domains.conll2003_ner.WIKIDATA_SMALL)
annotator = skweak.gazetteers.GazetteerAnnotators("wikismall", tries)

annotator(doc)
display_entities(doc, "wikismall_*")

Extracting data from ./data/wikidata_small.json
Populating trie for class PERSON (number: 1865712)
Populating trie for class LOC (number: 14250)
Populating trie for class GPE (number: 273743)
Populating trie for class ORG (number: 91421)
Populating trie for class PRODUCT (number: 12457)


As we can see, the "cased" gazetteers have a higher precision than the uncased gazetteers (at a cost of lower coverage).

#### 2.2 Crunchbase

The second gazetteer is extracted from the [Open Data Map from Crunchbase](https://data.crunchbase.com/docs/open-data-map), which contains lists of both organisations and (business) persons.

In [9]:
tries = skweak.gazetteers.extract_json_data(domains.conll2003_ner.CRUNCHBASE)
annotator = skweak.gazetteers.GazetteerAnnotators("crunchbase", tries)

annotator(doc)
skweak.utils.display_entities(doc, "crunchbase_*")

Extracting data from ./data/crunchbase.json
Populating trie for class COMPANY (number: 788915)
Populating trie for class ORG (number: 263)
Populating trie for class PERSON (number: 1062669)


#### 2.3 Geonames

The [geonames](http:www.geonames.org) database contains a large list of locations, including both geopolitical entities and "natural" locations:

In [9]:
tries = skweak.gazetteers.extract_json_data(domains.conll2003_ner.GEONAMES)
annotator = skweak.gazetteers.GazetteerAnnotators("geo", tries)

annotator(doc)
skweak.utils.display_entities(doc, "geo_*")

Extracting data from ./data/geonames.json
Populating trie for class GPE (number: 15205)


#### 2.4 Company names

Yet another gazetteer comes from the database of company names used internally at Exabel.  For each company name listed in this database, we extract possible alternatives, for instance with or without legal type (e.g. "Microsoft" vs. "Microsoft Corp.") or with or without country name ("SAS Norway" -> "SAS") or other generic tokens ("Nordea Bank" -> "Nordea"). As the database is quite large, the annotator takes some time to load:

In [17]:

tries = skweak.gazetteers.extract_json_data(domains.conll2003_ner.COMPANY_NAMES)
annotator = skweak.gazetteers.GazetteerAnnotators("companies", tries)

annotator(doc)
skweak.utils.display_entities(doc, "companies_*")

Extracting data from ./data/company_names.json
Populating trie for class COMPANY (number: 8678139)
Populating trie for class ORG (number: 254295)


Note that the annotator explicitly marks the detected entities with the label `COMPANY` instead of the more generic `ORG`.  

#### 2.5 Product names

Finally, I used [DBPedia](http://www.dbpedia.org) to extract a list of products and brands, since the recognition of products is particularly poor in Spacy NER models:

In [10]:

tries = skweak.gazetteers.extract_json_data(domains.conll2003_ner.PRODUCTS)
annotator = skweak.gazetteers.GazetteerAnnotators("products", tries)

annotator(doc)
skweak.utils.display_entities(doc, "products_*")

Extracting data from ./data/products.json
Populating trie for class PRODUCT (number: 45362)


### 3. Shallow patterns

Some named entities can also be captured through relatively simple, handcrafted patterns defined on the Spacy document. The class `FunctionAnnotator` makes it easy to define an annotator based on a function that takes a Spacy document as input and generate text spans with a label. Relations of mutual exclusivity between annotation sources can also be specified in the annotator. For instance, we can specify that numbers that are part of a date, time or money span should be ignored from the "number_detector" (to avoid having e.g. the `21` in `October 21` labelled as a `CARDINAL`): 

In [11]:

date_annotator = skweak.heuristics.FunctionAnnotator("date_detector", domains.conll2003_ner.date_generator)
time_annotator = skweak.heuristics.FunctionAnnotator("time_detector", domains.conll2003_ner.time_generator)
money_annotator = skweak.heuristics.FunctionAnnotator("money_detector", domains.conll2003_ner.money_generator)
exclusives = ["date_detector", "time_detector", "money_detector"]
number_annotator = skweak.heuristics.FunctionAnnotator("number_detector", domains.conll2003_ner.number_generator)
number_annotator.add_incompatible_sources(exclusives)

date_annotator(doc)
time_annotator(doc)
money_annotator(doc)
number_annotator(doc)
skweak.utils.display_entities(doc, "date_detector")
skweak.utils.display_entities(doc, "time_detector")
skweak.utils.display_entities(doc, "money_detector")
skweak.utils.display_entities(doc, "number_detector")

I have also created a range of patterns aiming to improve the _detection_ of named entities, even though they leave the actual label underspecified (as a generic `ENT` label). Four such detectors are constructed:
- two detectors of proper names based on casing (marking sequence of tokens whose lemma are "titled" as potential named entities)
- one detector of NNP sequences (based on the Spacy POS tagger)
- and one detector of sequences with proper names linked with "compound" dependency relations

In [12]:
# Detection based on casing
proper_detector = skweak.heuristics.TokenConstraintAnnotator("proper_detector", skweak.utils.is_likely_proper, "ENT")
    
# Detection based on casing, but allowing some lowercased tokens
proper2_detector = skweak.heuristics.TokenConstraintAnnotator("proper2_detector", skweak.utils.is_likely_proper, "ENT")
proper2_detector.add_gap_tokens(domains.conll2003_ner.LOWERCASED_TOKENS | domains.conll2003_ner.NAME_PREFIXES)
        
# Detection based on part-of-speech tags
nnp_detector = skweak.heuristics.TokenConstraintAnnotator("nnp_detector", lambda tok: tok.tag_=="NNP", "ENT")
        
# Detection based on dependency relations (compound phrases)
compound = lambda tok: skweak.utils.is_likely_proper(tok) and skweak.utils.in_compound(tok)
compound_detector = skweak.heuristics.TokenConstraintAnnotator("compound_detector", compound, "ENT")
 
combined = skweak.base.CombinedAnnotator()
exclusives = ["date_detector", "time_detector", "money_detector"]
for annotator in [proper_detector, proper2_detector, nnp_detector, compound_detector]:
    annotator.add_incompatible_sources(exclusives)
    annotator.add_gap_tokens(["'s", "-"])
    combined.add_annotator(annotator)

    # We add one variants for each NE detector, looking at infrequent tokens
    infrequent_name = "infrequent_%s"%annotator.name
    combined.add_annotator(skweak.heuristics.SpanConstraintAnnotator(infrequent_name, annotator.name, skweak.utils.is_infrequent))

doc = combined(doc)
skweak.utils.display_entities(doc, "proper_detector")
skweak.utils.display_entities(doc, "proper2_detector")
skweak.utils.display_entities(doc, "nnp_detector")
skweak.utils.display_entities(doc, "compound_detector")

Finally, I created three specific annotators:
- to recognise company names with a legal type
- full person names (with a first name along a list of common first names)
- slightly less common entities such as `NORP`, `FAC`, `LANGUAGE`, `EVENT` and `LAW`.

In [13]:

# Other types (legal references etc.)      
misc_detector = skweak.heuristics.FunctionAnnotator("misc_detector", domains.conll2003_ner.misc_generator)
legal_detector = skweak.heuristics.FunctionAnnotator("legal_detector", domains.conll2003_ner.legal_generator)
        
# Detection of companies with a legal type
ends_with_legal_suffix = lambda x: x[-1].lower_.rstrip(".") in domains.conll2003_ner.LEGAL_SUFFIXES
company_type_detector = skweak.heuristics.SpanConstraintAnnotator("company_type_detector", "proper2_detector", 
                                                    ends_with_legal_suffix, "COMPANY")
# Detection of full person names
full_name_detector = skweak.heuristics.SpanConstraintAnnotator("full_name_detector", "proper2_detector", 
                                                     domains.conll2003_ner.FullNameDetector(), "PERSON")

legal_detector(doc)
company_type_detector(doc)
full_name_detector(doc)
misc_detector(doc)
skweak.utils.display_entities(doc, "company_type_detector")
skweak.utils.display_entities(doc, "full_name_detector")
skweak.utils.display_entities(doc, "misc_detector")
skweak.utils.display_entities(doc, "legal_detector")

Finally, we also rely on an external probabilistic [parser of named entities](https://github.com/snipsco/snips-nlu-parsers) from [Snips](https://snips.ai/). The parser recognises `DATE`, `TIME`, `ORDINAL`, `CARDINAL`, `MONEY` and `PERCENT`. The parser is implemented in _Rust_, so it runs quite fast.

In [14]:
# Detection based on a probabilistic parser
snips = domains.conll2003_ner.SnipsAnnotator("snips")
snips(doc)
skweak.utils.display_entities(doc, "snips")

### 4. Document-level annotators

All annotators presented so far rely on _local_ decisions on tokens or phrases.  However, news articles are not mere collections of words, but exhibit a high degree of internal coherence. This can be exploited to furhter improve the annotation. Two document-level annotators are implemented:

Before we can run the document-level annotators, we need to normalise some of the entities. The `StandardiseAnnotator` is responsible for this normalisation:
- entities `PER` (from conll2003, BTC and SEC) are set to `PERSON`
- entities `LOC` from conll2003, BTC and SEC for spans that are also annotated by other layers as `GPE` are set to `GPE` 
- entities `ORG` that are annotated by other layers as `COMPANY` are set to `COMPANY`
    

In [15]:
from domains.conll2003_ner import ConLL2003Standardiser
annotator = ConLL2003Standardiser()
doc = annotator(doc)

In [16]:
from skweak.labelling import MajorityVoter
from skweak.utils import display_entities

mv = MajorityVoter("mv", ["LOC", "MISC", "ORG", "PER"])
mv.add_constraint_label("ENT", {"LOC", "MISC", "ORG", "PER"})
doc = mv(doc)
display_entities(doc, "mv")

#### 4.1 Document history

When a journalist first mentions an entity such as a company or person in an article, they typically write it in a "long form", and then use shorter mentions once the entity is properly introduced. For instance, in the text above, "Scott Moore" is first mentioned with a full name, and then simply referred to as "Moore". Similarly, companies are often first introduced to with their legal type.  The `DocumentHistoryAnnotator` takes advantage of this property, by propagating the label from the first mention onto subsequent mentions:

In [17]:
from skweak.doclevel import DocumentHistoryAnnotator
from skweak.utils import display_entities

annotator = DocumentHistoryAnnotator("doc_history", "mv", ["PER", "ORG"])
annotator(doc)
display_entities(doc, "doc_history_org_cased")

#### 4.2 Label consistency

Another property of news documents is the fact that two (or more) named entities sharing the same string in a text typically refer to the same entity, and should therefore have the same label. "Komatsu" can be both a company name and a city in Japan, but within a given document, it will typically be one or the other for the whole document. We can capture this fact with an annotator that looks at the majority label for a given string, and annotate all occurrences with this label:

In [18]:
from skweak.doclevel import DocumentMajorityAnnotator
from skweak.utils import display_entities

annotator = DocumentMajorityAnnotator("doc_majority", "mv")
annotator(doc)
display_entities(doc, "doc_majority_org_cased")

In [20]:
from skweak.labelling import MajorityVoter
from skweak.utils import display_entities

mv = MajorityVoter("mv", ["LOC", "MISC", "ORG", "PER"])
mv.add_constraint_label("ENT", {"LOC", "MISC", "ORG", "PER"})
doc = mv(doc)
display_entities(doc, "mv")

In [ ]:
for source in doc.user_data["spans"]:
    for (start, end), label in doc.user_data["spans"][source].items():
        for pattern in ["Moore", "needs"]:
            if doc[start:end].text==pattern:
                print("%s on %s (%i:%i): %s"%(source, pattern, start,end,label))

## __Step 2__: Estimation of label model

We can construct a full annotator with all annotators described above, and then run it on a dataset such as Reuters, Bloomberg, or Acquire:

In [1]:
import skweak.aggregation, domains.conll2003_ner
#full_annotator = domains.conll2003_ner.NERAnnotator().add_shallow().add_doc_level()
full_annotator = domains.conll2003_ner.NERAnnotator().add_all()
print("Total number of annotators:", len(full_annotator.annotators))

Loading shallow functions
Loading Spacy NER models
Loading gazetteer supervision modules
Extracting data from ./data/wikidata_small.json
Populating trie for class PERSON (number: 1865712)
Populating trie for class LOC (number: 14250)
Populating trie for class GPE (number: 273743)
Populating trie for class ORG (number: 91421)
Populating trie for class PRODUCT (number: 12457)
Extracting data from ./data/geonames.json
Populating trie for class GPE (number: 15205)
Extracting data from ./data/crunchbase.json
Populating trie for class COMPANY (number: 788915)
Populating trie for class ORG (number: 263)
Populating trie for class PERSON (number: 1062669)
Extracting data from ./data/products.json
Populating trie for class PRODUCT (number: 45362)
Loading document-level supervision sources
Total number of annotators: 63


We can then take the raw data from reuters, run Spacy on the textual content, and finally apply the annotator to get annotations from the each source:

In [ ]:
# We annotate 200 documents, and store them in a Spacy DocBin file
full_annotator.annotate_docbin("./data/reuters.docbin", "./data/reuters_small.docbin", cutoff=200)
#full_annotator.annotate_docbin("./data/reuters_small.docbin", "./data/reuters_small.docbin")

One this is done, we can finally estimate a unified annotator model through weak supervision. The basic idea is to describe the named entity recognition problem as a _Hidden markov Model_ where the observations are the annotations from each source, and the states correspond to the "true" (hidden) labels for each token, as illustrated below.

<img src="hmm.png">

Since we don't have access to the true labels for each token, we will rely on _Baum-Welch_ (a variant of EM) to estimate the HMM model through unsupervised training. More specifically, we will need to estimate 3 models:
- the initial probabilities $P(Y_0)$ of the labels for the first token of a document
- the transition matrix $P(Y_i | Y_{i-1})$ for the labels 
- the emission models $P(\lambda_{i,j} | Y_i)$ of observing a particular value $\lambda_{i,j}$ (say, `B-PER`) from the source $j$ given the true label $Y_i$. In the current model, we assume the emissions to be independent of one another given the true label, to reduce the complexity of the model.

Given an annotated dataset, the HMM model can be easily estimated:

In [6]:
import skweak.aggregation

unified_model = skweak.aggregation.HMM("hmm", ["LOC", "MISC", "ORG", "PER"])
unified_model.add_constraint_label("ENT", ["LOC", "MISC", "ORG", "PER"])
# We then run Baum-Welch on the model (can take some time)
unified_model.fit("../data/reuters_small.docbin")

# Saving the model to a file
unified_model.save("../data/hmm_reuters_small.pkl")

Using the following sources: ['BTC', 'BTC_truecase', 'companies_company_cased', 'companies_company_uncased', 'companies_org_cased', 'companies_org_uncased', 'company_type_detector', 'compound_detector', 'core_web_md', 'core_web_md_truecase', 'crunchbase_company_cased', 'crunchbase_company_uncased', 'crunchbase_org_cased', 'crunchbase_org_uncased', 'crunchbase_person_cased', 'crunchbase_person_uncased', 'date_detector', 'doc_history_org_cased', 'doc_history_org_uncased', 'doc_history_per_cased', 'doc_history_per_uncased', 'doc_majority_loc_cased', 'doc_majority_loc_uncased', 'doc_majority_misc_cased', 'doc_majority_misc_uncased', 'doc_majority_org_cased', 'doc_majority_org_uncased', 'doc_majority_per_cased', 'doc_majority_per_uncased', 'full_name_detector', 'geo_gpe_cased', 'geo_gpe_uncased', 'infrequent_compound_detector', 'infrequent_nnp_detector', 'infrequent_proper2_detector', 'infrequent_proper_detector', 'legal_detector', 'misc_detector', 'money_detector', 'multitoken_companies_co

Note that the HMM model relies on some informative priors to facilitate the parameter estimation:
- the prior for the initial probabilities is a Dirichlet based on counts for the most reliable model (chosen right now to be the Spacy NER model trained on Ontonotes)
- the prior for the transition matrix is a list of Dirichlet also based on counts from the standard Spacy NER model.
- finally, the initial emission models are calculated based on subjective estimates of the relative precision and recall of each source. For instance, we know that a source like `company_type_detector` (which looks at legal suffixes such as "Inc." at the end of the noun phrase) has a very high precision, but a low recall , since many mentions of companies do not include a suffix. In contrast, gazeteers will tend to have a better recall, but a lower precision (some company names also happen to be names of geopolitical entities or persons).  The initial precisions and recalls provided to the model is specified in `SOURCE_PRIORS` in the file `labelling.py`. When a precision and recall is not provided for a given source, they are assumed to be zeros (for instance, `company_type_detector` only detects `COMPANY` entities and nothing else).  

One the model is learned, we can apply it as any other "annotator" object:

In [7]:
unified_model.pretty_print() 

HMM model on following sources: ['BTC', 'BTC_truecase', 'companies_company_cased', 'companies_company_uncased', 'companies_org_cased', 'companies_org_uncased', 'company_type_detector', 'compound_detector', 'core_web_md', 'core_web_md_truecase', 'crunchbase_company_cased', 'crunchbase_company_uncased', 'crunchbase_org_cased', 'crunchbase_org_uncased', 'crunchbase_person_cased', 'crunchbase_person_uncased', 'date_detector', 'doc_history_org_cased', 'doc_history_org_uncased', 'doc_history_per_cased', 'doc_history_per_uncased', 'doc_majority_loc_cased', 'doc_majority_loc_uncased', 'doc_majority_misc_cased', 'doc_majority_misc_uncased', 'doc_majority_org_cased', 'doc_majority_org_uncased', 'doc_majority_per_cased', 'doc_majority_per_uncased', 'full_name_detector', 'geo_gpe_cased', 'geo_gpe_uncased', 'infrequent_compound_detector', 'infrequent_nnp_detector', 'infrequent_proper2_detector', 'infrequent_proper_detector', 'legal_detector', 'misc_detector', 'money_detector', 'multitoken_companies

In [9]:
import skweak.utils
reader = skweak.utils.docbin_reader("../data/reuters_small.docbin")
doc = next(reader)
#full_annotator(doc)
unified_model(doc)
skweak.utils.display_entities(doc, "hmm")
doc2 = next(reader)
#full_annotator(doc2)
unified_model(doc2)
skweak.utils.display_entities(doc2, "hmm")

In [ ]:
for source in unified_model.sources:
    print(source, ":", {(doc[start:end],start):label for (start,end), label in doc.user_data["spans"].get(source, {}).items() if doc[start:end].text=="Moore"})

And we can apply it to the full dataset:

In [19]:
unified_model.annotate_docbin("./data/reuters_small.docbin", "./data/reuters_small.docbin")

Reading ./data/reuters_small.docbin...Finished annotating ./data/reuters_small.docbin
Write to ./data/reuters_small.docbin...done


<br>

## __Step 3__: Development of neural NER model


We can now learn a neural NER model based on these unified annotations. We have two options: a straighforward (but slightly underperforming) approach using Spacy, and a more sophisticated approach using our own NER model

### __Alternative 1__: Using Spacy

In [3]:
import skweak.utils
skweak.utils.json_writer(skweak.utils.docbin_reader("../data/reuters_small.docbin", cutoff=20), "../data/reuters_small_dev.json")
skweak.utils.json_writer(skweak.utils.docbin_reader("../data/reuters_small.docbin", nb_to_skip=20),  "../data/reuters_small_train.json")


Writing JSON file to ../data/reuters_small_dev.json
Writing JSON file to ../data/reuters_small_train.json


And we can then directly train a new NER model with Spacy's training regime:

In [6]:
!rm -rf ../data/reuters_small_spacy
import spacy
spacy.cli.train(lang="en", output_path="../data/reuters_small_spacy", 
                train_path="../data/reuters_small_train.json", 
                dev_path="../data/reuters_small_dev.json",
               base_model="en_core_web_md", vectors="en_core_web_md",
               pipeline="ner", n_iter=10, replace_components=True)

✔ Created output directory: ../data/reuters_small_spacy
Training pipeline: ['ner']
Starting with base model 'en_core_web_md'
Replacing component from base model 'ner'
Loading vector from model 'en_core_web_md'
Counting training words (limit=0)
  0%|          | 0/88545 [00:00<?, ?it/s]
Itn  NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  ------  ------  ------  -------  -------
  0%|          | 389/88545 [00:00<00:31, 2831.04it/s]  9   1447.448  85.015  83.483  84.242   99.981    20918
 10   1274.511  85.191  83.784  84.481   99.981    18818
✔ Saved model to output directory
../data/reuters_small_spacy/model-final
✔ Created best model
../data/reuters_small_spacy/model-best


We then load the learned model (taking the model version with best performance on the development set):

In [7]:
nlp = spacy.load("../data/reuters_small_spacy/model-best")

And we finally run it on the document:

In [9]:
import skweak.utils
skweak.utils.display_entities(nlp(news_text))

NameError: name 'news_text' is not defined

# __STEP 3__: Evaluation

## Evaluation on ConLL 2003

We evaluate the approach on a standard dataset, namely conLL 2003. Note that ConLL only makes use of 4 labels (`PER`, `ORG`, `LOC`, `MISC`) instead of the 18 labels which were used above.


And we can then apply the NER model to annotate CoNLL 2003:

In [5]:
model.annotate_docbin("data/conll2003.docbin", "data/conll2003.docbin")

Finished annotating data/conll2003.docbin
Write to data/conll2003.docbin...done


The file `analysis.py` contains code to easily extract evaluation metrics by comparing the annotations from a particular annotation layer (for instance the HMM predictions, or the predictions from a single source) to the gold standard:

In [7]:
import analysis, annotations

# Extractint the documents
docs = list(annotations.docbin_reader("data/conll2003.docbin"))

# Here we extract for illustration purposes the metrics for two layers: the neural NER model trained
# on Ontonotes and the HMM predictions
analysis.evaluate(docs, ["ontonotes_ner", "HMM"], analysis.CONLL_MAPPINGS, analysis.CONLL_TO_RETAIN)

token_precision  token_recall  token_f1  \
label    proportion model                                                    
LOC      30.3 %     HMM                      0.720         0.864     0.786   
                    ontonotes_ner            0.666         0.849     0.746   
MISC     14.4 %     HMM                      0.807         0.579     0.674   
                    ontonotes_ner            0.846         0.439     0.578   
ORG      26.6 %     HMM                      0.538         0.677     0.600   
                    ontonotes_ner            0.628         0.373     0.468   
PER      28.7 %     HMM                      0.852         0.952     0.900   
                    ontonotes_ner            0.778         0.919     0.842   
macro               HMM                      0.729         0.768     0.748   
                    ontonotes_ner            0.730         0.645     0.684   
micro               HMM                      0.713         0.802     0.754   
                    ontonotes_ner            0.720         0.681     0.700   
weighted            HMM                      0.722         0.798     0.758   
                    ontonotes_ner            0.714         0.683     0.698   

                                   entity_precision  entity_recall  entity_f1  
label    proportion model                                                      
LOC      30.3 %     HMM                       0.734          0.841      0.784  
                    ontonotes_ner             0.656          0.835      0.734  
MISC     14.4 %     HMM                       0.827          0.570      0.674  
                    ontonotes_ner             0.785          0.512      0.620  
ORG      26.6 %     HMM                       0.536          0.532      0.534  
                    ontonotes_ner             0.433          0.269      0.332  
PER      28.7 %     HMM                       0.815          0.828      0.822  
                    ontonotes_ner             0.676          0.794      0.730  
macro               HMM                       0.728          0.693      0.710  
                    ontonotes_ner             0.638          0.602      0.620  
micro               HMM                       0.714          0.716      0.714  
                    ontonotes_ner             0.638          0.626      0.632  
weighted            HMM                       0.718          0.716      0.716  
                    ontonotes_ner             0.621          0.626      0.624